<table>
  <tr>
      <td><div style="white-space: pre-wrap;" align="left"><font size="30" >Tutorial B2&#10;Motion Model </font></div></td>
     <td><img src="https://github.com/Tobias-Fischer/RVSS2022/blob/main/Spatial_Awareness/Tutorial_B2_Robot_Localisation/image/RVSS-logo.png?raw=1" width="400"></td>
  </tr>
</table>

## Imports

In [2]:
#Import all the necessary modules
import sys


import matplotlib.pyplot as plt


#import import_ipynb
import math
import numpy as np
import pickle
#widgest allows interactive elements 
from ipywidgets import *

%matplotlib --list
#%matplotlib ipympl

!git clone https://github.com/Tobias-Fischer/RVSS2022.git
os.chdir('RVSS2022/Spatial_Awareness/Tutorial_B2_Robot_Localisation')

# use wget to obtain all the files we need for operation from github as a single tar file and extract here
#!wget --no-cache "https://raw.githubusercontent.com/Tobias-Fischer/RVSS2022/main/Spatial_Awareness/Tutorial_B2_Robot_Localisation/datalump.tar"
#!tar xf datalump.tar

#Visualizer
#from Renderer import *
#Here we import Robot class definition form the other notebook
from MotionModelRobot import *
#Here we import a class that defines out measurments
from Measurements import *

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'wx', 'qt4', 'qt5', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'inline', 'ipympl', 'widget']
Cloning into 'RVSS2022'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (409/409), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 409 (delta 189), reused 258 (delta 68), pack-reused 0
Receiving objects: 100% (409/409), 30.16 MiB | 11.32 MiB/s, done.
Resolving deltas: 100% (189/189), done.


# 2. PenguinPi Kinematic Model


The state of the robot is described by its 2D position<!---$Latex Expression$ allows us to incorporate Latex Equations into the document---> $(x,y)$  and orientation $\theta$. Our control inputs are number of encoder ticks of the left and right wheels of the robot. We can convert the encoder ticks into linear $v$ and angular $w$ velocities of the robot.

We will create a <code>drive(self)</code> function that will update the current state of the robot<code>self.state=</code>$(x_k,y_k,\theta_k)$ to the next state $(x_{k+1},y_{k+1},\theta_{k+1})$ given control inputs.

The control inputs are provided through the sliders that will appear once a PenguinPi object is created

In [3]:
class PenguinPi(object):
    
    def __init__(self, wheels_width, wheels_scale):
        # State is a vector of [x,y,theta]'
        self.state = np.zeros((3,1))
        
        # Wheel parameters
        self.wheels_width = wheels_width  # The distance between the left and right wheels
        self.wheels_scale = wheels_scale  # The scaling factor converting ticks/s to m/s
            
    def drive(self, drive_meas):
        # left_speed and right_speed are the speeds in ticks/s of the left and right wheels.
        # dt is the length of time to drive for

        # Compute the linear and angular velocity from wheel speeds
        linear_velocity, angular_velocity = self.convert_wheel_speeds(drive_meas.left_speed, drive_meas.right_speed)

        #This is the current state of the robot
        x_k = self.state[0]
        y_k = self.state[1]
        theta_k = self.state[2]
        
        # Apply the velocities
        dt = drive_meas.dt
        if angular_velocity == 0:
#-----------------------------FILL OUT DRIVE STRAIGHT CODE-----------------------------
#
            x_kp1 = x_k + np.cos(theta_k)*linear_velocity*dt
            y_kp1 = y_k + np.sin(theta_k)*linear_velocity*dt
            theta_kp1 = theta_k
#
#--------------------------------------------------------------------------------------
        else:
#--------------------------------FILL OUT DRIVE CODE-----------------------------------
#
            R = linear_velocity / angular_velocity
            theta_kp1 = theta_k + angular_velocity*dt
            x_kp1 = x_k + R * (np.sin(theta_kp1) - np.sin(theta_k))
            y_kp1 = y_k + R * (-np.cos(theta_kp1) + np.cos(theta_k))
#
#--------------------------------------------------------------------------------------
       
        #Save our state 
        self.state[0] = x_kp1
        self.state[1] = y_kp1
        self.state[2] = theta_kp1
                        
            
    def convert_wheel_speeds(self, left_speed, right_speed):
        # Convert to m/s
        left_speed_m = left_speed * self.wheels_scale
        right_speed_m = right_speed * self.wheels_scale

        # Compute the linear and angular velocity
        linear_velocity = (left_speed_m + right_speed_m) / 2.0
        angular_velocity = (right_speed_m - left_speed_m) / self.wheels_width
        
        return linear_velocity, angular_velocity

## Test Driving Straight

In [4]:
#Test our drive function driving straight
#Creater a new robot object with wheel width of 15cm and 1cm per wheel tick
bot = Robot(0.15, 0.01)
#Place the robot at -2.8,-2.8 which is bottom middle of our arena
bot.state = np.array([[-2.8],[-2.8],[np.pi/4]])
#This is our control signal
#Create a measurement object to drive the robot in a straight line
# Set robot velocity
left_speed, right_speed = 10, 10
# Set the sampling rate
dt = 0.2
drive_meas = DriveMeasurement(left_speed, right_speed, dt)

N = 200
state  = np.zeros((N,3))
for c in range(state.shape[0]):
    state[c,:] = bot.state[:,0]
    bot.drive(drive_meas)
#Create a visualizer and visualize our state
#rend = Rendered.Instance()
#rend.initialize(state)

In [10]:
import plotly.graph_objects as go
from PIL import Image

from google.colab import output
output.enable_custom_widget_manager()

from ipywidgets import widgets



# Create figure
#fig = go.Figure()

#fig.update_xaxes(showgrid=False, range = [-4, -2])
#fig.update_yaxes(showgrid=False, range = [-3.5, -1.5])
# Set templates

# Draw the robot

data = [go.Scatter(x=[state[0,0]],y=[state[0,1]])]
layout = {'xaxis': {'range': [-4,-2]},
          'yaxis': {'range': [-3.5, -1.5]}}
fig = go.FigureWidget(data, layout)

fig.update_layout(template="plotly_white")
fig.update_layout(width = 500, height=530) #makes it look square on my screen

c, s = np.cos(state[0,2]), np.sin(state[0,2])
arrowlength=0.15

fig.add_annotation(
  x=state[0,0]+arrowlength*c,  # arrows' head
  y=state[0,1]+arrowlength*s,  # arrows' head
  ax=state[0,0],  # arrows' tail
  ay=state[0,1],  # arrows' tail
  xref='x',
  yref='y',
  axref='x',
  ayref='y',
  text='',  # if you want only the arrow
  showarrow=True,
  arrowhead=3,
  arrowsize=1,
  arrowwidth=1,
  arrowcolor='red',
  visible=True
)

fig.add_annotation(
  x=state[0,0]-arrowlength*s,  # arrows' head
  y=state[0,1]+arrowlength*c,  # arrows' head
  ax=state[0,0],  # arrows' tail
  ay=state[0,1],  # arrows' tail
  xref='x',
  yref='y',
  axref='x',
  ayref='y',
  text='',  # if you want only the arrow
  showarrow=True,
  arrowhead=3,
  arrowsize=1,
  arrowwidth=1,
  arrowcolor='green',
  visible=True
)

#print(fig)

marker_files = [filename for filename in os.listdir('./image') if filename.startswith("M")]
marker_world_width = 0.3
for i,filename in enumerate(marker_files):
  fprts = filename.split('_')
  xpos = float(fprts[1])
  ypos = float(fprts[2])
  #mp = np.array([float(fprts[1]),float(fprts[2])])
  #mi = cv2.imread('./image/'+filename)
  im = Image.open('image/' + filename)
  imsize = 0.3

  # Add images
  fig.add_layout_image(
      dict(
          source = im,
          xref="x",
          yref="y",
          x=xpos-imsize/2,
          y=ypos+imsize/2,
          sizex=imsize,
          sizey=imsize,
          sizing="contain",
          opacity=1.0,
          layer="below")
)

#fig.show()

fig.data[0].x=[state[50,0]]
fig.data[0].y=[state[50,1]]

timestep = widgets.IntSlider(
    value=0,
    min=0,
    max=N-1,
    step=1,
    description='Time:',
    continuous_update=False
)

def response(change):
  fig.data[0].x=[state[change.new,0]]
  fig.data[0].y=[state[change.new,1]]
  print(change.new)
  print(fig.data[0])
  #print(change)

timestep.observe(response, names='value')

widgets.VBox([fig, widgets.HBox([timestep])])


    'data': [{'type': 'scatter',
              'uid': '42fdf971-d012-4205-be07-0…

In [23]:
import plotly.graph_objects as go
import numpy as np
import plotly.figure_factory as ff
fig = ff.create_quiver([state[0,0]],[state[0,1]],[np.cos(state[0,2])],[np.sin(state[0,2])])
fig.update_xaxes(showgrid=False, range = [-4, -2])
fig.update_yaxes(showgrid=False, range = [-3.5, -1.5])
fig.update_layout(template="plotly_white")
fig.update_layout(width = 500, height=530) #makes it look square on my screen

frames = []
for k in range(1,50):
    fig_aux = ff.create_quiver([state[k,0]], [state[k,1]], [np.cos(state[k,2])],[np.sin(state[k,2])])
    fig.add_trace(figaux.data[0])
    fig.data[k].visible=False

#print(fig.data)
fig.show()


In [43]:
import plotly.graph_objects as go
import numpy as np
import plotly.figure_factory as ff

numdata = len(state)

# Create figure
fig = go.Figure()
#fig = ff.create_quiver([state[0,0]],[state[0,1]],[np.cos(state[0,2])],[np.sin(state[0,2])],line_color='red')
fig.update_xaxes(showgrid=False, range = [-4, -2])
fig.update_yaxes(showgrid=False, range = [-3.5, -1.5])
fig.update_layout(template="plotly_white")
fig.update_layout(width = 500, height=530) #makes it look square on my screen

frames = []
for k in range(numdata):
    figaux1 = ff.create_quiver([state[k,0]], [state[k,1]], [np.cos(state[k,2])],[np.sin(state[k,2])],line_color='red', name='')
    fig.add_trace(figaux1.data[0])
    figaux2 = ff.create_quiver([state[k,0]], [state[k,1]], [-np.sin(state[k,2])],[np.cos(state[k,2])],line_color='green', name='')
    fig.add_trace(figaux2.data[0])
    fig.data[2*k].visible=False
    fig.data[2*k+1].visible=False

fig.update_xaxes(showgrid=False, range = [-4, -2])
fig.update_yaxes(showgrid=False, range = [-3.5, -1.5])
fig.update_layout(showlegend=False)

# Make 0th arrow pair visible
fig.data[0].visible = True
fig.data[1].visible = True

# Create and add slider
steps = []
for i in range(numdata):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][2*i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][2*i+1] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

## Test Driving in a Circle

In [ ]:
#Test our drive function
#Place the robot at -2.8,-2.8 which is bottom middle of our arena
bot.state = np.array([[-2.8],[-2.8],[0]])

#This is our control signal
#Create a measurement object to drive the robot in a circle
# Set robot velocity
left_speed, right_speed = 10, 14
# Set the sampling rate
dt = 0.2
drive_meas = DriveMeasurement(left_speed, right_speed, dt)

N = 200
state  = np.zeros((N,3))
for c in range(state.shape[0]):
    state[c,:] = bot.state[:,0]
    bot.drive(drive_meas)

#Create a visualizer and visualize our state
rend = Rendered.Instance()
rend.initialize(state)


## Test Driving Real Data

In [ ]:
#Finally test our robot with some real data

#Here we load our data including the true state 
with open('data/data.dat', "rb") as f:
    data = pickle.load(f)
    
gt_state = data['state'][48:,:]
control = data['control'][48:,:]

bot = Robot(0.15, 0.01)
bot.state = np.array([[-2.8],[-2.8],[0]])
#Place the robot at the correct initial state X0
bot.state = gt_state[0,:].copy().reshape(-1,1)

#Apply our control to the robot and visualize
#Our timestamp is always 0.2
dt = 0.2
#Here we will save our state variable 
state = np.zeros((control.shape[0],3))

for c in range(control.shape[0]):
    state[c,:] = bot.state[:,0]
    drive_signal = DriveMeasurement(control[c,0],control[c,1],dt)
    bot.drive(drive_signal)

#Create a visualizer and visualize our state
rend = Rendered.Instance()
rend.initialize(state,gt_state = gt_state)


In [ ]:
gt_state[48:,:]

In [ ]:
time = np.arange(0,control.shape[0])*dt
plt.figure()
plt.plot(time,control[:,0])
plt.plot(time,control[:,1])
